In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Deepchem\\")

In [4]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [5]:
Pubchem_data=pd.read_csv("./Original_data/Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [6]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [7]:
#Load the data
Raw_data= pd.read_csv('./Original_data/Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./Original_data/X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Models/Scaler_transformer.pkl')

In [8]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Original_data/Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7261 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [9]:
X_scaled_data

,piID,piPC06,piPC05,piPC07,piPC04,piPC03,piPC08,piPC09,piPC02,piPC10,...,N-076,nArNH2,nRCO,SHED_DN,MAXDN,SssO,T(O..F),F01[C-N],N-069,JGT
mol_id,,,,,,,,,,,,,,,,,,,,,
TOX1002,0.281159,0.633935,0.648573,0.611759,0.635256,0.621807,0.485628,0.000000,0.604590,0.000000,...,0.333333,0.0,0.0,0.000000,0.007217,0.000000,0.000000,0.088235,0.0,0.539761
TOX10047,0.361336,0.710360,0.746295,0.664370,0.726682,0.706401,0.659917,0.650077,0.683573,0.640127,...,0.000000,0.0,0.0,0.000000,0.002711,0.064971,0.000000,0.000000,0.0,0.601982
TOX10056,0.351116,0.681562,0.724311,0.640111,0.707454,0.683773,0.641363,0.623419,0.662888,0.616667,...,0.000000,0.0,0.0,0.000000,0.002574,0.063855,0.000000,0.000000,0.0,0.555317
TOX1006,0.159590,0.000000,0.000000,0.000000,0.283839,0.376899,0.000000,0.000000,0.412851,0.000000,...,0.000000,0.0,0.0,0.000000,0.007265,0.000000,0.000000,0.088235,0.0,0.533730
TOX10060,0.118383,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.335031,0.000000,...,0.000000,0.0,0.0,0.000000,0.005208,0.000000,0.000000,0.000000,0.0,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOX9869,0.371412,0.682679,0.727091,0.640111,0.713603,0.709874,0.599545,0.563081,0.704932,0.547351,...,0.000000,0.0,0.0,0.141806,0.007768,0.059834,0.000000,0.058824,0.0,0.538967
TOX9874,0.376317,0.716361,0.744022,0.697034,0.731117,0.719857,0.674758,0.650077,0.707778,0.642199,...,0.000000,0.0,0.0,0.000000,0.006834,0.119761,0.000000,0.000000,0.0,0.592950
TOX9879,0.445144,0.761250,0.781350,0.733056,0.791829,0.777703,0.705248,0.683702,0.769399,0.670866,...,0.000000,0.0,0.0,0.000000,0.006724,0.205505,0.000000,0.323529,0.0,0.449605


In [10]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [12]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [13]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,piID,piPC06,piPC05,piPC07,piPC04,piPC03,piPC08,piPC09,piPC02,piPC10,...,N-076,nArNH2,nRCO,SHED_DN,MAXDN,SssO,T(O..F),F01[C-N],N-069,JGT
cid,,,,,,,,,,,,,,,,,,,,,
5273,8.287336,5.855475,5.522710,6.172108,5.080239,4.441180,6.472340,6.198177,3.766997,6.131456,...,0.0,1.0,0.0,0.0,1.256435,5.587037,0.0,10.0,1.0,0.323926
13463,8.761969,5.971621,5.620627,6.331738,5.169418,4.563045,6.676842,6.591599,3.917011,6.661040,...,0.0,1.0,0.0,0.0,2.446908,5.517269,0.0,10.0,1.0,0.468256
39148,8.936284,6.001763,5.649194,6.372693,5.213916,4.603919,6.728998,6.701125,3.974998,6.813020,...,0.0,1.0,0.0,0.0,2.161352,5.769213,0.0,13.0,1.0,0.471748
56328,8.662170,5.941199,5.598653,6.293203,5.146404,4.520429,6.622232,6.499517,3.876396,6.540697,...,0.0,1.0,0.0,0.0,2.545611,5.411944,8.0,10.0,1.0,0.446062
67358,8.863011,6.077678,5.734150,6.452702,5.266891,4.631082,6.781293,6.758534,3.974998,6.712087,...,0.0,1.0,0.0,4.0,2.639871,5.341991,0.0,10.0,1.0,0.493542


In [14]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./Original_data/X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7261 Moleculars × 584 Descriptors


,piID,piPC06,piPC05,piPC07,piPC04,piPC03,piPC08,piPC09,piPC02,piPC10,...,N-076,nArNH2,nRCO,SHED_DN,MAXDN,SssO,T(O..F),F01[C-N],N-069,JGT
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.353850,0.744712,0.744258,0.727205,0.741027,0.698061,0.706793,0.630984,0.648572,0.593916,...,0.0,0.25,0.0,0.000000,0.003624,0.061357,0.000000,0.294118,0.25,0.323926
13463,0.374116,0.759484,0.757453,0.746013,0.754035,0.717216,0.729126,0.671035,0.674400,0.645213,...,0.0,0.25,0.0,0.000000,0.007058,0.060590,0.000000,0.294118,0.25,0.468256
39148,0.381559,0.763317,0.761303,0.750838,0.760526,0.723640,0.734821,0.682185,0.684384,0.659935,...,0.0,0.25,0.0,0.000000,0.006235,0.063357,0.000000,0.382353,0.25,0.471748
56328,0.369855,0.755615,0.754492,0.741473,0.750678,0.710517,0.723162,0.661661,0.667407,0.633556,...,0.0,0.25,0.0,0.000000,0.007343,0.059434,0.021798,0.294118,0.25,0.446062
67358,0.378431,0.772972,0.772752,0.760265,0.768253,0.727910,0.740532,0.688030,0.684384,0.650158,...,0.0,0.25,0.0,0.283611,0.007615,0.058666,0.000000,0.294118,0.25,0.493542
